## Introducción e hipótesis

El objetivo es predecir si un cliente se dará de baja o no de la plataforma.

In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

## Carga de datos

In [ ]:
data  = pd.read_csv('./data/Datos ML 2021 Q2.csv', sep=';')
print("El dataset tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))
data.head()

## Descripción del dataset

#### El dataset tiene las siguientes columnas:

- CustomerID: ID del cliente
- Churn: Columna que indica si el cliente dejó de usar la plataforma o no. 1 es que se da de baja.
- CustomerTenure: Es el tiempo transcurrido desde el inicio de la relación con el cliente (en meses)
- MainDeviceLogin: Dispositivo principal que utiliza el cliente para acceder a la plataforma
- CityTier: Indicador del nivel de desarrollo de la ciudad donde vive el cliente
- WarehouseToHome: Distancia desde el centro de distribución a la vivienda del cliente (en km)
- MainPaymentMode: Método de pago más utilizado por el cliente
- Gender: Género del cliente
- HourSpendOnApp: Número de horas que el cliente ha pasado en la plataforma
- DeviceRegistered: Número de dispositivos en los que el cliente ha accedido a la plataforma
- PrefCategory: Categoría más común de las compras del cliente en el último mes
- SatisfactionScore: Nivel de satisfacción del cliente con el servicio
- MaritalStatus: Estado civil del cliente
- NumberOfAddress: Número de direcciones diferentes registradas por el cliente
- Complain: Si ha realizado reclamos
- OrderAmountHikeFromlastYear: Incremento porcentual en la cantidad de compras con respecto al año anterior
- CouponUsed: Número de cupones usados en el último mes
- OrderCount: Número de compras realizadas en el último mes
- DaySinceLastOrder: Cantidad de días desde la última compra
- CashbackAmount: Promedio de reembolsos pedidos en el último mes

## Exploración de datos

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
display(data['Churn'].value_counts())
display(data['Churn'].value_counts(normalize=True))

Analicemos las columnas sospechosas

In [ ]:
##Se consideran columnas con valores sospechosos aquellas cuya máxima que se encuentran 
# por encima de 3 desviaciones estándar de la media. 

std_limit = 3

##Por la naturaleza de las variables, se considera que los valores sospechosos son aquellos 
# que se encuentran por encima y no los inferiores.



In [ ]:
# Columnas sospechosas

suspicious_columns = []

for col in data.columns:
    if(data[col].dtype == 'object'):
        continue
    mean = data[col].mean()
    std = data[col].std()
    max = data[col].max()
    if(max > mean + std_limit*std):
        suspicious_columns.append(data[col].name)
        

suspicious_columns


In [ ]:
suspicious_rows_arr = []

def investigate_suspicious_column(data, column, watch_outliers=True):
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    plt.suptitle(column)
    sns.histplot(data[column], ax=ax[0])
    sns.boxplot(data=data[column], ax=ax[1], orient='h')

    plt.show()

    if(watch_outliers):
        mean = data[column].mean()
        std = data[column].std()
        max = data[column].max()

        suspicious_rows = data[data[column] > mean + std_limit*std]
        suspicious_rows_arr.append(suspicious_rows)
        display("Hay {} filas sospechosas".format(suspicious_rows.shape[0]))
        display(suspicious_rows)

In [ ]:
for col in suspicious_columns:
    investigate_suspicious_column(data, col)

In [ ]:
## Total de filas sospechosas

print("Hay {} filas sospechosas".format(sum([suspicious_rows.shape[0] for suspicious_rows in suspicious_rows_arr])))

In [ ]:
## Filas sospechosas agrupadas por columna churn

suspicious_rows = pd.concat(suspicious_rows_arr)
display(suspicious_rows['Churn'].value_counts())
display(suspicious_rows['Churn'].value_counts(normalize=True))


Observamos que la distribución de la variable Churn entre los valores extremos es similar, por lo que no parece haber una relación entre la variable y la variable objetivo.

Por otro lado, haciendo una observación pormenorizada, creemos que en los casos de las columnas `CouponUsed`, `OrderCount` y `DaySinceLastOrder`  y `CashbackAmount` parecen ser valores lógicos, aún tratándose de valores extremos por lo que no las eliminaremos.

In [ ]:
suspicious_columns

In [ ]:
suspicious_columns.remove('CouponUsed')
suspicious_columns.remove('OrderCount')
suspicious_columns.remove('DaySinceLastOrder')
suspicious_columns.remove('CashbackAmount')

suspicious_columns

Limpiemos las filas sospechosas

In [ ]:
def remove_outliers(data, column):
    mean = data[column].mean()
    std = data[column].std()
    max = data[column].max()
    return data[data[column] <= mean + std_limit*std]

In [ ]:
for col in suspicious_columns:
    data = remove_outliers(data, col)

Veamos el resultado

In [ ]:
for col in suspicious_columns:
    investigate_suspicious_column(data, col, False)

## Visualización de datos

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(32,32))
fig.suptitle('Histogramas normalizados')
for c, ax in zip(data.columns, axes.flatten()):
    sns.histplot(data = data.loc[data['Churn']==0, c].dropna(), stat = 'density', ax = ax, kde = False )
    sns.histplot(data = data.loc[data['Churn']==1, c].dropna(), stat = 'density', kde=False, ax=ax, color = 'orange')
    ax.legend(['Churn = 0', 'Churn = 1'])

Podemos inferir que las variables que tendrán una significancia en establecer el valor del churn son aquellas en las cuales podemos observar diferencias en la distribución de los valores de churn.

Por lo tanto podemos descartar las variables que no presentan diferencias en la distribución de los valores de churn.

In [ ]:
## Variables a eliminar

columns_to_eliminate = ['CustomerID', 'Gender', 'NumberOfAddress']

data = data.drop(columns_to_eliminate, axis=1)

Veamos la correlación entre las variables

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(data.corr(), annot=True, vmin=-1, cmap='Blues')

## Limpieza de datos

In [ ]:
## Veamos cuantos valores nulos hay en cada columna

data.isna().sum()

In [ ]:
## Los eliminamos

data.dropna(inplace=True)

display(data.isna().sum())

print("El dataset limpio tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))



Observemos la distribución de los valores de churn

In [ ]:
display(data['Churn'].value_counts())
display(data['Churn'].value_counts(normalize=True))

El dataset está desbalanceado, por lo que se deberá tener en cuenta al momento de entrenar los modelos.

En base a la proporción de valores de churn, nuestra hipótesis nula es 0.16 que es la proporción de valores de churn positivos (que se dará de baja) en el dataset.

## Separación de datos

In [ ]:
X = data.drop(['Churn'], axis=1)
y = data['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Preparación de datos

In [ ]:
categorical_columns = [col for col in data.columns if data[col].dtypes == 'object']

categorical_columns

In [ ]:
numerical_columns = [col for col in data.columns if data[col].dtypes != 'object']

numerical_columns

In [ ]:
# DeviceRegistered es una variable categórica 

numerical_columns.remove('DeviceRegistered')
categorical_columns.append('DeviceRegistered')

# La transformamos a categórica

X_train['DeviceRegistered'] = X_train['DeviceRegistered'].astype('object')
X_test['DeviceRegistered'] = X_test['DeviceRegistered'].astype('object')


In [ ]:
# Complain es una variable categórica binaria 

numerical_columns.remove('Complain')
categorical_columns.append('Complain')

# La transformamos a categórica

X_train['Complain'] = X_train['Complain'].astype('object')
X_test['Complain'] = X_test['Complain'].astype('object')

In [ ]:
display('categorical_columns',categorical_columns)
display('numerical_columns',numerical_columns)

### Variables categóricas

In [ ]:
encoder_categories = []

X_categorical_columns = [x for x in categorical_columns]

for col in X_categorical_columns:    
    col_categories = data[col].unique()
    encoder_categories.append(col_categories)

encoder_categories

In [ ]:
encoder = OneHotEncoder(categories = encoder_categories, sparse=False, drop='first')

encoder = encoder.fit(X_train[X_categorical_columns])

X_train_encoded = encoder.transform(X_train[X_categorical_columns])
X_train_categorical = pd.DataFrame(X_train_encoded, columns = encoder.get_feature_names_out(X_categorical_columns))

X_test_encoded = encoder.transform(X_test[X_categorical_columns])
X_test_categorical = pd.DataFrame(X_test_encoded, columns = encoder.get_feature_names_out(X_categorical_columns))
X_test_categorical.head()

### Variables numéricas

In [ ]:
X_train_numerical = X_train.drop(X_categorical_columns, axis=1)
X_test_numerical = X_test.drop(X_categorical_columns, axis=1)

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_numerical)
X_train_numerical = pd.DataFrame(X_train_scaled, columns = X_train_numerical.columns)

X_test_scaled = scaler.transform(X_test_numerical)
X_test_numerical = pd.DataFrame(X_test_scaled, columns = X_test_numerical.columns)
X_test_numerical.head()

Unimos las variables numéricas y categóricas

In [ ]:
X_train = pd.concat([X_train_categorical, X_train_numerical], axis=1)
X_test = pd.concat([X_test_categorical, X_test_numerical], axis=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train.head()

## KNN

### Modelo

### Métricas

## Regresión logística

### Modelo

### Métricas

## Naive Bayes

### Modelo

### Métricas

## Conclusiones